In [2]:
## Imports

# General
import os
import os.path as path
import json
from glob import glob
import cv2 as cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [3]:
# Set data to be processed
DROOT = './0_data'

# Sources
RND_SRC = path.join(DROOT, "2_procfootage/360-cropped.jpg")

# Destination video
VID_DST_FRMS = path.join(DROOT, "4_simsiam/video_sample/")
VID_DST = path.join(DROOT, "4_simsiam/video_sample.mp4")

In [4]:
base = plt.imread(RND_SRC)
height, width, layers = base.shape
print(base.shape)
r = 150

(1810, 6516, 3)


In [5]:
length = 100

def get_random_movement(size, main_dev, n_part, part_dev, start, bias):
    main = np.random.normal(0, main_dev, length)
    main_summed = np.array([sum(main[0:i]) + start + bias for i in range(size)])
    parts = [np.random.normal(main_summed, part_dev)
             for x in range(n_part)]
    
    return parts
    
combine_lists_of_lists = lambda x, y: [list(zip(x[i], y[i])) for i in range(len(x))]

part_1_x = get_random_movement(length, 400, 3, 100, -600, width/2)
part_1_y = get_random_movement(length, 20, 3, 20, -100, height/2)
part_1 = combine_lists_of_lists(part_1_x, part_1_y)

part_2_x = get_random_movement(length, 100, 4, 200, 400, width/2)
part_2_y = get_random_movement(length, 20, 4, 20, 100, height/2)
part_2 = combine_lists_of_lists(part_2_x, part_2_y)

part_3_x = get_random_movement(length, 400, 4, 100, 1000, width/2)
part_3_y = get_random_movement(length, 20, 4, 20, -200, height/2)
part_3 = combine_lists_of_lists(part_3_x, part_3_y)

part_4_x = get_random_movement(length, 100, 5, 150, -700, width/2)
part_4_y = get_random_movement(length, 20, 5, 20, 400, height/2)
part_4 = combine_lists_of_lists(part_4_x, part_4_y)

parts = pd.DataFrame(part_1 + part_2 + part_3 + part_4).T

parts.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"(2613.2344269833707, 800.7192381810094)","(2529.4122891494003, 819.5036700952793)","(2734.835115781451, 809.6951925678163)","(3624.1725792296133, 1000.710800374332)","(3416.378911676135, 1007.6283112483752)","(3213.230793522332, 979.5710351584648)","(3977.204449465545, 1000.4977199107858)","(4105.014764181258, 700.2142112410282)","(4168.307449235088, 669.9644783074699)","(4142.118030848721, 727.3886054422208)","(4272.388644854969, 663.2978064881493)","(2627.5107258539465, 1291.843719115189)","(2455.1341928900615, 1280.37677693771)","(2396.026807437184, 1301.1958361825014)","(2782.2881148890665, 1302.569297203743)","(2630.1077187085693, 1321.6475060200064)"


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'FMP4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video = cv2.VideoWriter(VID_DST, fourcc, 10, (1440, 720))

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)
cmap = get_cmap(len([x[0] for x in parts.iloc[0]]))

for frame in tqdm(parts.itertuples(index=False)):
    
    fig, ax = plt.subplots(figsize=(20,10))
    ax.imshow(base)
    x = [x[0] for x in frame]
    y = [x[1] for x in frame]
    ax.scatter(x, y, c=range(len(x)), cmap=cmap, linewidths=15, alpha=0.7)
    ax.axis('off')
    fig.tight_layout(pad=0, h_pad=0)
    ax.margins(0, y=0)

    fig.canvas.draw()
    image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    video.write(cv2.cvtColor(image_from_plot, cv2.COLOR_BGR2RGB))
    plt.close()
    
video.release()

94it [01:02,  1.52it/s]